### Sample Code - Load Transactions Function

In [9]:
import csv
from datetime import datetime

transactions = []

def load_transactions(filename = 'financial_transactions_short.csv'):
    ERROR_LOG_FILE = 'errors.txt'
    try:
        with open(ERROR_LOG_FILE, 'w') as f:
            f.write(f"[{datetime.now()}] --- Transaction Processing Log Started ---\n")
        print(f"Log file '{ERROR_LOG_FILE}' has been created/cleared.")
    except IOError as e:
        print(f"Failed to initialize error log file '{ERROR_LOG_FILE}': {e}")
      
    try:
        with open(filename, 'r', newline='') as file:
            csv_reader = csv.DictReader(file)
            for row in csv_reader:
                transactions.append(row)                
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return None
    
    for item in transactions:
        date_str = item['date']
        try:
            parsed_date = datetime.strptime(date_str, "%Y-%m-%d")
        except ValueError:
            try:
                parsed_date = datetime.strptime(date_str, "%d-%m-%Y")
            except ValueError:
                try:
                    parsed_date = datetime.strptime(date_str, "%m/%d/%Y")
                except ValueError:
                    parsed_date = None

        if parsed_date:
            item['date'] = parsed_date.strftime("%Y, %M, %d")
        else:
            with open(ERROR_LOG_FILE, 'a') as f:
                f.write(f"Error: Skipping transaction {item['transaction_id']}. Invalid date.\n")

    for item in transactions:
        try:
            if not item['amount'] or item['amount'].strip() =="":
                new_amount = 0.0
                print(f"Warning: Empty amount found for transaction {item['transaction_id']}. Setting to 0.0")
            else:
                new_amount = float(item['amount'])
            
            if item['type'] == "debit":
                item['amount'] = new_amount * -1
            elif item['type'] == "credit":
                item['amount'] = new_amount            
        except ValueError:
            with open(ERROR_LOG_FILE, 'a') as f:
                f.write(f"Error: Could not convert amount to float in transaction {item['transaction_id']}. Invalid amount.")



    print(f"There are {len(transactions)} transactions.")
    print(f"Transaction processing complete. Check {ERROR_LOG_FILE} for any logged issues.")

    return transactions

load_transactions()


Log file 'errors.txt' has been created/cleared.
There are 20 transactions.
Transaction processing complete. Check errors.txt for any logged issues.


[{'transaction_id': '1',
  'date': '2020, 00, 26',
  'customer_id': '926',
  'amount': 6478.39,
  'type': 'credit',
  'description': 'Expect series shake art again our.'},
 {'transaction_id': '2',
  'date': '2020, 00, 08',
  'customer_id': '466',
  'amount': 1255.95,
  'type': 'credit',
  'description': 'Each left similar likely coach take.'},
 {'transaction_id': '3',
  'date': '2019, 00, 02',
  'customer_id': '110',
  'amount': -7969.68,
  'type': 'debit',
  'description': 'Direction wife job pull determine leader move college.'},
 {'transaction_id': '4',
  'date': '2020, 00, 02',
  'customer_id': '142',
  'amount': 2927.41,
  'type': 'credit',
  'description': 'Agree reveal buy black already.'},
 {'transaction_id': '5',
  'date': '2020, 00, 02',
  'customer_id': '944',
  'amount': -4661.88,
  'type': 'debit',
  'description': 'Child relationship show college whom speech.'},
 {'transaction_id': '6',
  'date': '2021, 00, 25',
  'customer_id': '900',
  'amount': 3649.68,
  'type': 'cred

### Sample Code - Add Transactions Function

In [10]:
import csv
from datetime import datetime

def add_transactions(filename = 'financial_transactions_short.csv'):
    transactions = []
    max_id = 0
    try:
        with open(filename, 'r', newline='') as file:
            csv_reader = csv.DictReader(file)
            for row in csv_reader:
                transactions.append(row)
                current_id = int(row['transaction_id'])
                if current_id > max_id:
                    max_id = current_id                
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return None
   
    print("--- Add New Transaction ---")
    
    transaction_id = max_id + 1

    while True:
        date_str = input("Enter date (YYYY-MM-DD): ").strip()
        try:
            datetime.strptime(date_str, '%Y-%m-%d')
            break
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD.")

    customer_id = input("Enter customer ID: ")

    while True:
        amount_str = input("Enter amount: ").strip()
        try:
            amount = float(amount_str)
            break
        except ValueError:
            print("Invalid amount. Please enter numerical value.")

    type = input("Enter type (credit/debit/transfer): ").strip()
    if not type:
        print("Transaction type cannot be empty. Aborting...")
        return
      
    description = input("Enter description for transaction: ").strip()

    try:
        new_transaction = {'transaction_id': transaction_id, 'date': date_str, 'customer_id': customer_id, 'amount': amount, 'type': type, 'description': description}
        transactions.append(new_transaction)
        print(f"Enter date (YYYY-MM-DD): {date_str}")
        print(f"Enter customer ID: {customer_id}")
        print(f"Enter amount: {amount}")
        print(f"Enter type (credit/debit/transfer): {type}")
        print(f"Enter description: {description}")
        print(f"Transaction added!")
        print(transactions[-1])
    except IOError as e:
        print(f"Error writing to file '{filename}': {e}")

add_transactions()


--- Add New Transaction ---
Enter date (YYYY-MM-DD): 2025-06-01
Enter customer ID: 645
Enter amount: 85.0
Enter type (credit/debit/transfer): credit
Enter description: credit
Transaction added!
{'transaction_id': 21, 'date': '2025-06-01', 'customer_id': '645', 'amount': 85.0, 'type': 'credit', 'description': 'credit'}


### Sample Code - View Transactions Function

In [12]:
import csv
from datetime import datetime

transactions = []

def view_transactions(filename = 'financial_transactions_short.csv'):
    ERROR_LOG_FILE = 'errors.txt'
    try:
        with open(ERROR_LOG_FILE, 'w') as f:
            f.write(f"[{datetime.now()}] --- Transaction Processing Log Started ---\n")
        print(f"Log file '{ERROR_LOG_FILE}' has been created/cleared.")
    except IOError as e:
        print(f"Failed to initialize error log file '{ERROR_LOG_FILE}': {e}")
    
    try:
        with open(filename, 'r', newline='') as file:
            csv_reader = csv.DictReader(file)
            for row in csv_reader:
                transactions.append(row)                
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return None
    
    columns = [('transaction_id', 'ID', 6), ('date', 'Date', 12), ('customer_id', 'Customer', 10),
               ('amount', 'Amount', 12), ('type', 'Type', 10), ('description', 'Description', 40)]
    
    header_parts = []
    separator_parts = []
    for key, display_name, width in columns:
        header_parts.append(f"{display_name:<{width}}")
        separator_parts.append("-" * width)
    print(" | ".join(header_parts))
    print(" | ".join(separator_parts))

    for transaction in transactions:
        row_values = []
        for key, _, width in columns:
            value = transaction.get(key, '')

            if key == 'amount':
                try:
                    value = f"{float(value):.2f}"
                except (ValueError, TypeError):
                    value = "N/A"

            str_value = str(value)

            if len(str_value) > width:
                str_value = str_value[:width - 3] + "..."

            row_values.append(f"{str_value:<{width}}")

        print(" | ".join(row_values))

view_transactions()

Log file 'errors.txt' has been created/cleared.
ID     | Date         | Customer   | Amount       | Type       | Description                             
------ | ------------ | ---------- | ------------ | ---------- | ----------------------------------------
1      | 10/26/2020   | 926        | 6478.39      | credit     | Expect series shake art again our.      
2      | 1/8/2020     | 466        | 1255.95      | credit     | Each left similar likely coach take.    
3      | 9/2/2019     | 110        | 7969.68      | debit      | Direction wife job pull determine lea...
4      | 12/2/2020    | 142        | 2927.41      | credit     | Agree reveal buy black already.         
5      | 12/2/2020    | 944        | 4661.88      | debit      | Child relationship show college whom ...
6      | 4/25/2021    | 900        | 3649.68      | credit     | Century front item.                     
7      | 6/19/2020    | 458        | 504.24       | credit     | Market cover loss hotel send.          

### Sample Code - Update Transactions Function

In [13]:
import csv
from datetime import datetime

transactions = []

def update_transactions(filename='financial_transactions_short.csv'):
    ERROR_LOG_FILE = 'errors.txt'
    try:
        with open(ERROR_LOG_FILE, 'w') as f:
            f.write(f"[{datetime.now()}] --- Transaction Processing Log Started ---\n")
        print(f"Log file '{ERROR_LOG_FILE}' has been created/cleared.")
    except IOError as e:
        print(f"Failed to initialize error log file '{ERROR_LOG_FILE}': {e}")
    
    try:
        with open(filename, 'r', newline='') as file:
            csv_reader = csv.DictReader(file)
            for row in csv_reader:
                transactions.append(row)                
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return None

    if transactions is None:
        print("Cannot update transactions as no data was loaded.")
        return
    
    print("\nUpdating:")

    columns = [('transaction_id', 'ID', 6), ('date', 'Date', 12), ('customer_id', 'Customer', 10),
               ('amount', 'Amount', 12), ('type', 'Type', 10), ('description', 'Description', 40)]
    
    header_parts = []
    separator_parts = []
    for key, display_name, width in columns:
        header_parts.append(f"{display_name:<{width}}")
        separator_parts.append("-" * width)
    print(" | ".join(header_parts))
    print(" | ".join(separator_parts))

    for transaction in transactions:
        row_values = []
        for key, _, width in columns:
            value = transaction.get(key, '')

            if key == 'amount':
                try:
                    value = f"{float(value):.2f}"
                except (ValueError, TypeError):
                    value = "N/A"

            str_value = str(value)

            if len(str_value) > width:
                str_value = str_value[:width - 3] + "..."

            row_values.append(f"{str_value:<{width}}")

        print(" | ".join(row_values))

    while True:
        try:
            selection = int(input(f"Select transaction (1-{len(transactions)}): "))
            if 1 <= selection <= len(transactions):
                selected_transaction_index = selection - 1
                break
            else:
                print(f"Invalid selection. Please enter a number between 1 and {len(transactions)}")
        except ValueError:
            print("Invalid input. Please enter a number.")

    selected_transaction = transactions[selected_transaction_index]

    while True:
        field_to_change = input("Change which field? (description, type, amount): ").strip().lower()
        if field_to_change in ['description', 'type', 'amount']:
            break
        else:
            print("Invalid field. Please choose from 'description', 'type', or 'amount'.")
    
    new_value = input(f"New {field_to_change}: ").strip()

    if field_to_change == 'amount':
        try:
            float(new_value)
        except ValueError:
            print("Invalid amount. Please enter a numeric value.")
            return
    
    selected_transaction[field_to_change] = new_value

    try:
        if transactions:
            fieldnames = transactions[0].keys()
        else:
            fieldnames = [col[0] for col in columns]
        
        with open(filename, 'w', newline='') as file:
            csv_writer = csv.DictWriter(file, fieldnames=fieldnames)
            csv_writer.writeheader()
            csv_writer.writerows(transactions)
        print(f"Select transaction (1-{len(transactions)}): {selection}")
        print(f"Change which field? (description, type, amount): {field_to_change}")
        print(f"New {field_to_change}: {new_value}")
        print("Transaction updated!")
    except IOError as e:
        print(f"Error writing updated transactions to '{filename}': {e}")
    except Exception as e:
        print(f"An unexpected error occurred during file write: {e}")

update_transactions()

Log file 'errors.txt' has been created/cleared.

Updating:
ID     | Date         | Customer   | Amount       | Type       | Description                             
------ | ------------ | ---------- | ------------ | ---------- | ----------------------------------------
1      | 10/26/2020   | 926        | 6478.39      | credit     | Expect series shake art again our.      
2      | 1/8/2020     | 466        | 1255.95      | credit     | Each left similar likely coach take.    
3      | 9/2/2019     | 110        | 7969.68      | debit      | Direction wife job pull determine lea...
4      | 12/2/2020    | 142        | 2927.41      | credit     | Agree reveal buy black already.         
5      | 12/2/2020    | 944        | 4661.88      | debit      | Child relationship show college whom ...
6      | 4/25/2021    | 900        | 3649.68      | credit     | Century front item.                     
7      | 6/19/2020    | 458        | 504.24       | credit     | Market cover loss hotel send

### Sample Code - Delete Transactions Function

In [9]:
import csv
from datetime import datetime

transactions = []

def delete_transactions(filename='financial_transactions_short.csv'):
    ERROR_LOG_FILE = 'errors.txt'
    try:
        with open(ERROR_LOG_FILE, 'w') as f:
            f.write(f"[{datetime.now()}] --- Transaction Processing Log Started ---\n")
        print(f"Log file '{ERROR_LOG_FILE}' has been created/cleared.")
    except IOError as e:
        print(f"Failed to initialize error log file '{ERROR_LOG_FILE}': {e}")
    
    try:
        with open(filename, 'r', newline='') as file:
            csv_reader = csv.DictReader(file)
            for row in csv_reader:
                transactions.append(row)                
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return None

    if not transactions:
        print("Cannot delete transactions as no data was loaded.")
        return
    
    print("\nTransactions available for deletion:")

    columns = [('transaction_id', 'ID', 6), ('date', 'Date', 12), ('customer_id', 'Customer', 10),
               ('amount', 'Amount', 12), ('type', 'Type', 10), ('description', 'Description', 40)]
    
    header_parts = []
    separator_parts = []
    for key, display_name, width in columns:
        header_parts.append(f"{display_name:<{width}}")
        separator_parts.append("-" * width)
    print(" | ".join(header_parts))
    print(" | ".join(separator_parts))

    for i, transaction in enumerate(transactions):
        row_values = []
        transaction_display_id = i + 1
        row_values.append(f"{transaction_display_id:<{columns[0][2]}}")
        
        for key, _, width in columns[1:]:
            value = transaction.get(key, '')

            if key == 'amount':
                try:
                    value = f"{float(value):.2f}"
                except (ValueError, TypeError):
                    value = "N/A"

            str_value = str(value)

            if len(str_value) > width:
                str_value = str_value[:width - 3] + "..."

            row_values.append(f"{str_value:<{width}}")

        print(" | ".join(row_values))

    while True:
        try:
            selection = int(input(f"Select transaction to delete (1-{len(transactions)}): "))
            if 1 <= selection <= len(transactions):
                selected_transaction_index = selection - 1
                break
            else:
                print(f"Invalid selection. Please enter a number between 1 and {len(transactions)}")
        except ValueError:
            print("Invalid input. Please enter a number.")

    selected_transaction = transactions[selected_transaction_index]

    print("\nYou have selected the following transaction for deletion: ")
    print("-" * 60)
    for key, display_name, _ in columns:
        value = selected_transaction.get(key, 'N/A')
        if key == 'amount':
            try:
                value = f"{float(value):.2f}"
            except (ValueError, TypeError):
                value = "N/A"
        print(f"{display_name}: {value}")
    print("-" * 60)

    confirm = input("Are you sure you want to delete this transaction? (yes/no): ").strip().lower()

    if confirm == 'yes':
        del transactions[selected_transaction_index]
        try:
            if transactions:
                fieldnames = transactions[0].keys()
            else:
                fieldnames = [col[0] for col in columns]
            with open(filename, 'w', newline='') as file:
                csv_writer = csv.DictWriter(file, fieldnames=fieldnames)
                csv_writer.writeheader()
                csv_writer.writerows(transactions)
            print(f"Select transaction (1 - {len(transactions)}): {selection}")
            print(f"Are you sure? (yes/no): {confirm}")
            print("Transaction deleted successfully!")
        except IOError as e:
            print(f"Error writing updated transactions to '{filename}': {e}")
        except Exception as e:
            print(f"An unexpected error occurred during file write: {e}")
    else:
        print("Transaction deletion cancelled.")

delete_transactions()

Log file 'errors.txt' has been created/cleared.

Transactions available for deletion:
ID     | Date         | Customer   | Amount       | Type       | Description                             
------ | ------------ | ---------- | ------------ | ---------- | ----------------------------------------
1      | 10/26/2020   | 926        | 6478.39      | transfer   | Expect series shake art again our.      
2      | 1/8/2020     | 466        | 1255.95      | credit     | Each left similar likely coach take.    
3      |              | 110        | 7969.68      | credit     | Direction wife job pull determine lea...
4      | 12/2/2020    | 142        | 2927.41      | credit     | Agree reveal buy black already.         
5      | 12/2/2020    | 944        | 4661.88      | debit      | Child relationship show college whom ...
6      | 4/25/2021    | 900        | 3649.68      | credit     | Century front item.                     
7      | 6/19/2020    | 458        | 504.24       | credit     | M

In [5]:
import csv
from datetime import datetime

transactions = []
ERROR_LOG_FILE = 'errors.txt'

def _log_error(message):
    with open(ERROR_LOG_FILE, 'a') as f:
        f.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {message}\n")

def load_transactions(filename='financial_transactions_short.csv'):
    global transactions    

    try:
        with open(filename, 'r', newline='') as file:
            csv_reader = csv.DictReader(file)
            for row in csv_reader:
                processed_row = row.copy()

                date_str = processed_row.get('date', '').strip()
                parsed_date = None
                date_formats = ["%Y-%m-%d", "%d-%m-%Y", "%m/%d/%Y"]

                if date_str:
                    for fmt in date_formats:
                        try:
                            parsed_date = datetime.strptime(date_str, fmt)
                            break
                        except ValueError:
                            continue
                    if parsed_date:
                        processed_row['date'] = parsed_date
                    else:
                        _log_error(f"Error: Skipping transaction {processed_row.get('transaction_id', 'N/A')}. Invalid date format '{date_str}.")
                        processed_row['date'] = None

                    try:
                        amount_str = processed_row.get('amount', '').strip()
                        new_amount = 0.0
                        if not amount_str:
                            _log_error(f"Warning: Empty amount found for transaction {processed_row.get('transaction_id', 'N/A')}. Setting to 0.0.")
                        else:
                            new_amount = float(amount_str)
                        transaction_type = processed_row.get('type', '').strip().lower()
                        if transaction_type == "debit":
                            processed_row['amount'] = new_amount * -1
                        elif transaction_type == 'credit':
                            processed_row['amount'] = new_amount
                        else:
                            processed_row['amount'] - new_amount
                            if transaction_type:
                                _log_error(f"Warning: Unrecognizable type '{transaction_type} for transaction {processed_row.get('transaction_id', 'N/A')}. Amount stored as is.")
                    except (ValueError, TypeError):
                        _log_error(f"Error: Could not convert amount to float in transaction {processed_row.get('transaction_id', 'N/A')}. Invalid amount '{processed_row.get('amount', '')}'.")
                        processed_row['amount'] = 0.0

                    transactions.append(processed_row)

    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        _log_error(f"Error: The file '{filename}' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred during file loading: {e}")
        _log_error(f"An unexpected error occurred during file loading: {e}")
    
    print(f"There are {len(transactions)} transactions.")
    print(f"Transaction processing complete. Check {ERROR_LOG_FILE} for any logged issues.")

def add_transactions(transactions):
    load_transactions()
    print(transactions)
    max_id = 0
    
    for transaction in transactions:
        if 'transaction_id' in transaction:
            try:
                current_id = int(transaction['transaction_id'])
                if current_id > max_id:
                    max_id = current_id
            except (ValueError, TypeError):
                print(f"Warning: transaction_id '{transaction['transaction_id']}' is not a valid number. Skipping.")
        else:
            print("Warning: 'transaction_id' key not found in transaction.")
   
    print("--- Add New Transaction ---")
    
    transaction_id = max_id + 1

    while True:
        date_str = input("Enter date (YYYY-MM-DD): ").strip()
        try:
            datetime.strptime(date_str, '%Y-%m-%d')
            break
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD.")

    customer_id = input("Enter customer ID: ")

    while True:
        amount_str = input("Enter amount: ").strip()
        try:
            amount = float(amount_str)
            break
        except ValueError:
            print("Invalid amount. Please enter numerical value.")

    type = input("Enter type (credit/debit/transfer): ").strip()
    if not type:
        print("Transaction type cannot be empty. Aborting...")
        return
      
    description = input("Enter description for transaction: ").strip()

    try:
        new_transaction = {'transaction_id': transaction_id, 'date': date_str, 'customer_id': customer_id, 'amount': amount, 'type': type, 'description': description}
        transactions.append(new_transaction)
        print(f"Enter date (YYYY-MM-DD): {date_str}")
        print(f"Enter customer ID: {customer_id}")
        print(f"Enter amount: {amount}")
        print(f"Enter type (credit/debit/transfer): {type}")
        print(f"Enter description: {description}")
        print(f"Transaction added!")
        print(transactions[-1])
    except IOError as e:
        print(f"Unable to add transaction.")

add_transactions(transactions)

There are 20 transactions.
Transaction processing complete. Check errors.txt for any logged issues.
[{'transaction_id': '1', 'date': datetime.datetime(2020, 10, 26, 0, 0), 'customer_id': '926', 'amount': 6478.39, 'type': 'credit', 'description': 'Expect series shake art again our.'}, {'transaction_id': '2', 'date': datetime.datetime(2020, 1, 8, 0, 0), 'customer_id': '466', 'amount': 1255.95, 'type': 'credit', 'description': 'Each left similar likely coach take.'}, {'transaction_id': '3', 'date': datetime.datetime(2019, 9, 2, 0, 0), 'customer_id': '110', 'amount': -7969.68, 'type': 'debit', 'description': 'Direction wife job pull determine leader move college.'}, {'transaction_id': '4', 'date': datetime.datetime(2020, 12, 2, 0, 0), 'customer_id': '142', 'amount': 2927.41, 'type': 'credit', 'description': 'Agree reveal buy black already.'}, {'transaction_id': '5', 'date': datetime.datetime(2020, 12, 2, 0, 0), 'customer_id': '944', 'amount': -4661.88, 'type': 'debit', 'description': 'Chil

In [29]:
transactions = []

def view_transactions(transactions):
    if not transactions:
        print(f"No transactions to display. Load or add some transactions.")
        return
    
    columns = [('transaction_id', 'ID', 6), ('date', 'Date', 12), ('customer_id', 'Customer', 10),
               ('amount', 'Amount', 12), ('type', 'Type', 10), ('description', 'Description', 40)]
    
    header_parts = []
    separator_parts = []
    for key, display_name, width in columns:
        header_parts.append(f"{display_name:<{width}}")
        separator_parts.append("-" * width)
    print(" | ".join(header_parts))
    print(" | ".join(separator_parts))

    for transaction in transactions:
        row_values = []
        for key, _, width in columns:
            value = transaction.get(key, '')

            if key == 'amount':
                try:
                    value = f"{float(value):.2f}"
                except (ValueError, TypeError):
                    value = "N/A"

            str_value = str(value)

            if len(str_value) > width:
                str_value = str_value[:width - 3] + "..."

            row_values.append(f"{str_value:<{width}}")

        print(" | ".join(row_values))

view_transactions(transactions)

No transactions to display. Load or add some transactions.


### Main Program - Smart Personal Finance Analyzer

In [ ]:
import csv
from datetime import datetime
import personal_finance_lib5
transactions = []

def main():
    while True:
        print("\nSmart Personal Finance Analyzer")
        print("1. Load Transactions")
        print("2. Add Transaction")
        print("3. View Transactions")
        print("4. Update Transaction")
        print("5. Delete Transaction")
        print("6. Analyze Finances")
        print("7. Save Transactions")
        print("8. Generate Report")
        print("9. Exit")
        choice = input("Select an option: ")
        # Call functions based on choice
        if choice == '9':
            break
        elif choice == '1':
            load_transactions()
        elif choice == '2':
            personal_finance_lib5.add_transactions(transactions)
        elif choice == '3':
            view_transactions(transactions)
        elif choice == '4':
            personal_finance_lib5.delete_transactions(transactions)


main()